<a href="https://colab.research.google.com/github/blue-create/langlens/blob/main/to_publish/rule_based_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regelbasiertes Filtern
In diesem Notebook werden Zeitungsartikel nach Schlagworten und RegEx-Ausdrücken nach verschiedenen Kategorien gefiltert.

Schritte:
- Filtern nach Sensationalismus
- Filtern nach Graphisch
- Filtern nach Statistiken
- Filtern nach Hilfsangebote

### Imports, Konstanten, Paths

In [ ]:
import tqdm as tqdm
import os
import pandas as pd
import re
from ast import literal_eval
import re
import pandas as pd
from google.colab import drive

In [ ]:
#Definieren der Paths
folder_path = "2_Filtered"

In [ ]:
# Verbinden mit GDrive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/data/

### Daten


In [ ]:
# Lesen und zusammenfügen aller Dateien
dfs = []
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, index_col=0, converters={"text":literal_eval})
        dfs.append(df)
df_filtered = pd.concat(dfs, ignore_index=True)

In [ ]:
#Testen der Filter mit kleinerem Datensatz
df_test=df.iloc[:5000,:]
df_test=df_test.explode("text")

## Filtern nach Sensationalismus

Die Liste von RegEx-Ausdrücken und Schlagwörtern wurde in einem iterativen Prozess erstellt um möglichste viele Artikel mit sensationalistischer Sprache zu erhalten.

In [ ]:
# Definieren von Schlagworten, die auf sensationalistische Texte hindeuten können
sens =["Ehetragödie",
       "verschmähter? Liebhaber(in)?",
       "rachsüchtige Hexe",
       "Eifersuchtstat",
       "Beziehungsdramas?",
       "Rosenkrieg",
       "Frauenschläger",
       "tödliches?m? Drama",
       "tränenerstickter? Stimme",
       "blutüberströmt",
       "tödlichen? Eskalation",
       "ehedrama",
]

In [ ]:
# Funktion zum prüfen von Texten auf sensationalistische Schlagworte
def label_sensationalism(text):
  """ Funktion zum prüfen von Texten auf sensationalistische Schlagworte
  Parameters:
    - text (str): Text, der nach sensationalistischen Schlagworten überprüft werden soll
  Returns:
    - str: "Sensationalist" wenn sensationalistische Schlagworte in dem Text auftauchen, "" wenn nicht
  """
  for pattern in sens:
    if len(re.findall(pattern=pattern,string=text,flags=re.IGNORECASE))>0:
      return "Sensationalist"
  return ""

In [ ]:
# Kategorisieren des Testsets
df_test["label"]=df_test.text.apply(label_sensationalism)

## Filtern nach Graphischer Sprache

Diese Liste von Schlagworten beinhalten Ausdrücke, die auf verstörende Inhalte hinweisen.

In [ ]:
# Definieren von Schlagworten, die auf graphische Inhalte hindeuten können
GRAPHIC=["vergewaltigt","vergewaltigung","sexuell missbraucht"]

In [ ]:
# Funktion zum prüfen von Texten auf graphische Schlagworte
def label_graphic(text):
  """ Funktion zum prüfen von Texten auf Schlagworte, die auf graphische Inhalte hinweisen
  Parameters:
    - text (str): Text, der auf Schlagworte überprüft werden soll
  Returns:
    - str: "Graphic" wenn Schlagworte in dem Text auftauchen, "" wenn nicht
  """
  for pattern in GRAPHIC:
    if len(re.findall(pattern=pattern,string=text,flags=re.IGNORECASE))>0:
      return "Graphic"
  return ""

In [ ]:
# Kategorisieren des Testsets
df_test["label"]=df_test.text.apply(label_graphic)

## Filtern nach Statistiken

Die Liste von RegEx-Ausdrücken und Schlagwörtern wurde in einem iterativen Prozess erstellt um möglichste viele Artikel mit Statistiken zu erhalten


In [ ]:
# Definieren von RegEx-Ausdrücken um Statistiken zu finden
stats=["\d*([\.,]\d*)?[ ](Prozent|%)",
       "\d+ (Fälle|Opfer|Frauen|Kinder|Personen|Betroffene|Morde|Plätze)\W",
       "Bundeskriminalamt",
       "Kriminalstatistik",
       "statistisches Amt",
       "offizielle Zahlen",
       "jeder? (zweiter?|dritter?|vierter?|fünfter?|sechster?|siebenter?) (mann|frau|kind)",
       "(eine|zwei|drei|vier|fünf) von (zwei|drei|vier|fünf) Frauen",
       "\Wbka\W"]

In [ ]:
#Funktion zum prüfen von Texten auf Statistiken
def label_statistics(text):
  """ Funktion zum prüfen von Texten auf Statistiken
  Parameters:
    - text (str): Text, der auf Statistiken überprüft werden soll
  Returns:
    - str: "Statistic" wenn Statisiken in dem Text auftauchen, "" wenn nicht
  """
  for pattern in stats:
    if len(re.findall(pattern=pattern,string=text,flags=re.IGNORECASE))>0:
      return "Statistic"
  return ""

In [ ]:
# Kategorisieren des Testsets
df_test["label"]=df_test.text.apply(label_statistics)

## Filtern nach Hilfsangeboten

Die Liste von RegEx-Ausdrücken und Schlagwörtern wurde in einem iterativen Prozess erstellt um möglichste viele Artikel mit Statistiken zu erhalten

In [ ]:
# Definieren von RegEx-Ausdrücken um Hilfsangebote zu finden
support=["[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}",
         "([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+"
         '\(?\d{4,5}\)?[/\s]*\d{1,5}\s*\d{1,5}',
         "(www\.)\S+\.\S+",
         "Beratungsstelle",
         "Notrufdienst",
         "Hotline",
         "\d{2,4}\s?\(?\d{1,5}\)?[-.\s/]?(\d{1,8}[-.\s/]?){1,5}\d{2,8}"
         ]

In [ ]:
#Funktion zum Prüfen von Texten auf Hilfsangebote
def label_support(text):
  """ Funktion zum Prüfen von Texten auf Hilfsangebote
  Parameters:
    - text (str): Text, der auf Hilfsangebote überprüft werden soll
  Returns:
    - str: "Support Service" wenn Hilfsangebote in dem Text auftauchen, "" wenn nicht
  """
  for pattern in support:
    if len(re.findall(pattern=pattern,string=text,flags=re.IGNORECASE))>0:
      return "Support Service"
  return ""

In [ ]:
# Kategorisieren des Testsets
df_test["label"]=df_test.text.apply(label_support)